In [7]:
using Profile
using PProf
using Plots
using Distributions
include("zipf.jl")

mutable struct quant_OGD_integral{TP<:Integer,TC<:UInt8} # TP = Type Pointer, TC = Type Counter
    const C_UPPER::TP
    cache_size::TP # tracks the current cache size
    const ONE::TC
    lazy_update::TC
    step_size::TC
    count::TC # for count lookup and calculation
    d::TC # for putting the d calculation
    u::TC # for uniform lookup 
    counter_uniform::Dict{TP,Tuple{TC,TC}} # pointer -> (counter, uniform)
    d_look::Dict{TC,Set{TP}} # d -> set of pointers with this d
    val_look::Dict{TC,Set{TP}} # counter -> set of pointers with this counter
    to_evict::Set{TP} # set of pointers of delayed evictions
end


function Base.show(io::IO, q::quant_OGD_integral)
    println(io, "QuantOGD Parameters:")
    println(io, "-------------------------")
    println(io, "C_UPPER:    ", q.C_UPPER)
    println(io, "Cache Size: ", q.cache_size)
    println(io, "ONE:        ", q.ONE)
    println(io, "Lazy Update: ", q.lazy_update)
    println(io, "Step Size:  ", q.step_size)
    println(io, "Counter:    ", q.count)
    println(io, "d:          ", q.d)
    println(io, "u:          ", q.u)
    println(io, "counter_uniform: ", q.counter_uniform)
    println(io, "d_look:     ", q.d_look)
    println(io, "val_look:   ", q.val_look)
    println(io, "to_evict:   ", q.to_evict)
    println(io, "-------------------------")
end


# Constructor for quantOGD
function init_quant_OGD_integral(;
    C=10,
    step_size=1,
    TP::Type{<:Integer}=UInt32,
    TC::Type{<:UInt8}=UInt8
)
    C_UPPER = TP(C)
    cache_size = zero(C_UPPER)
    ONE = TC(typemax(TC))
    lazy_update = zero(ONE)
    step_size = TC(step_size)
    count = zero(ONE)  # maybe I should init to nothing? 
    d = zero(ONE)
    u = zero(ONE)

    counter_uniform = Dict{TP,Tuple{TC,TC}}()
    d_look = Dict(TC(i) => Set{TP}() for i in 0:ONE)
    val_look = Dict(TC(i) => Set{TP}() for i in 0:ONE)
    to_evict = Set{TP}()

    return quant_OGD_integral{TP,TC}(C_UPPER, cache_size, ONE, lazy_update, step_size, count, d, u, counter_uniform, d_look, val_look, to_evict)
end


# UInt comparisons and additions should be checked carefully, we're not an 
# expert in auto promotions in julia

function step!(q::quant_OGD_integral, i::Integer)
    if haskey(q.counter_uniform, i) # checks if count[i] is non zero
        q.count, q.u = q.counter_uniform[i]
        delete!(q.val_look[q.count], i)
        if q.count - q.lazy_update > q.u # checks if d>0 
            q.d = q.count  - q.u
            delete!(q.d_look[q.d], i)
            q.cache_size -= one(q.cache_size)
        end

        if (q.ONE - q.step_size >= q.count - q.lazy_update) #checks if + step size will be bigger then 1 in mod 2^l
            q.count += q.step_size
        else
            q.count = q.lazy_update + q.ONE
        end

    else
        q.count = q.lazy_update + q.step_size
        # updating the uniform always would requiring to change the whole control flow
        q.u = rand(zero(q.ONE):q.ONE)
    end
    q.counter_uniform[i] = (q.count, q.u)

    push!(q.val_look[q.count], i)
    if q.count - q.lazy_update > q.u # checks if d>0 
        q.d = q.count - q.u
        # q.d_look[q.d] = push!(get(q.d_look, q.d, Set()), i)
        push!(q.d_look[q.d], i) # better but ensure that all q.d keys exists
        q.cache_size += one(q.cache_size)
        if (i in q.to_evict)
            delete!(q.to_evict, i)
            q.cache_size -= one(q.cache_size)
        end
    end

    resize_cache!(q)
end


function resize_cache!(q::quant_OGD_integral)
    while q.cache_size > q.C_UPPER
        if !isempty(q.to_evict)
            pop!(q.to_evict)
            q.cache_size -= one(q.cache_size)
        else
            q.lazy_update += one(q.lazy_update)
            # union!(q.to_evict, get(q.d_look, q.lazy_update, Set()))
            union!(q.to_evict, q.d_look[q.lazy_update]) # better but ensure that all q.d keys exists
            empty!(q.d_look[q.lazy_update])
            delete!.(Ref(q.counter_uniform), q.val_look[q.lazy_update]) # deletes all 0 counters from the dict
            empty!(q.val_look[q.lazy_update])
        end
    end
end


get_fraction(q::quant_OGD_integral, i) = (haskey(counter_uniform, i)) ? (q.counter[i][1] - q.lazy_update) / q.ONE : 0


get_fraction (generic function with 1 method)

In [9]:
q = init_quant_OGD_integral(C=800,step_size = UInt8(10))
step!(q, UInt32(10000))
println(q)

ii = repeat(UInt32(0):UInt32(999),100)
println(length(ii))
for i in ii
    step!(q, i)
end
cache = Set()
for (_, s) in q.d_look
    union!(cache,s)
end
union!(cache, q.to_evict)
println(q)
println("length of cache: ",length(cache))
println("cache: ",cache)

# println("counter uniform: ", q.counter_uniform)
# println("dlook: ", q.d_look)

QuantOGD Parameters:
-------------------------
C_UPPER:    800
Cache Size: 1
ONE:        255
Lazy Update: 0
Step Size:  10
Counter:    10
d:          7
u:          3
counter_uniform: Dict{UInt32, Tuple{UInt8, UInt8}}(0x00002710 => (0x0a, 0x03))
d_look:     Dict{UInt8, Set{UInt32}}(0x38 => Set(), 0x23 => Set(), 0x3c => Set(), 0xdc => Set(), 0x43 => Set(), 0xd7 => Set(), 0x49 => Set(), 0xfb => Set(), 0x73 => Set(), 0x70 => Set(), 0xb9 => Set(), 0x56 => Set(), 0xa8 => Set(), 0xcf => Set(), 0xf2 => Set(), 0xb7 => Set(), 0xe0 => Set(), 0xb1 => Set(), 0x0c => Set(), 0x4b => Set(), 0x17 => Set(), 0x6f => Set(), 0x29 => Set(), 0x44 => Set(), 0x52 => Set(), 0x82 => Set(), 0x7d => Set(), 0x4d => Set(), 0xac => Set(), 0x47 => Set(), 0x42 => Set(), 0x67 => Set(), 0x3b => Set(), 0xd0 => Set(), 0x1a => Set(), 0xd3 => Set(), 0x7f => Set(), 0x74 => Set(), 0x64 => Set(), 0x4f => Set(), 0xe6 => Set(), 0xc3 => Set(), 0x8d => Set(), 0x87 => Set(), 0x8a => Set(), 0xde => Set(), 0x6b => Set(), 0x2e => Set()

In [6]:
a= Dict{UInt8, Tuple{UInt8,UInt8}}()
a[1] = (2,3) 
a[2] = (4,5) 
println(a)

# a[1] = (1,1) 
# a[2] = (2,2) 
# println(a)
x,y = a[1]
println(y,x)


Dict{UInt8, Tuple{UInt8, UInt8}}(0x02 => (0x04, 0x05), 0x01 => (0x02, 0x03))
32


In [2]:
b = UInt8(5)
c = UInt8(254)
c = UInt8(255)
println(c)
println(b-c)
c += one(c)
println(c)
println(typeof(c))
println(typemax(c))

255
6
0
UInt8
255


In [27]:
c = Set([1,2,3,6,7,8])
println(c)
delete!.(Ref(c),Set([7,8]))
println(c)
delete!(c, 2)
println(c)
delete!(c, 4)
println(c)
pop!(c)
println(c)
empty!(c)
println(c)

Set([6, 7, 2, 8, 3, 1])
Set([6, 2, 3, 1])
Set([6, 3, 1])
Set([6, 3, 1])
Set([3, 1])
Set{Int64}()
